<a href="https://colab.research.google.com/github/daisy-pacheco/MLProject/blob/master/MERF_shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file_train = "/content/drive/My Drive/Colab Notebooks/num_train_data.csv"
file_test = "/content/drive/My Drive/Colab Notebooks/num_test_data.csv"

Mounted at /content/drive


In [38]:
train = pd.read_csv(file_train)
test = pd.read_csv(file_test)

In [39]:
# most average person



MERF

In [40]:
!pip install -q merf

In [41]:
from merf import MERF

In [42]:
def prep_data(data):
  data['Z'] = [1] * len(data)
  data['cluster'] = data['id'].str.replace(' _', '').astype(int)
  X = data[data.columns.difference(['job_satisfaction', 'id', 'Z', 'cluster', 'Unnamed: 0', 'employer_id'])]
  Z = data[['Z']]
  clusters = data['cluster']
  y = data['job_satisfaction']
  return X, Z, clusters, y

In [43]:
X_train, Z_train, clusters_train, y_train = prep_data(train)
X_test, Z_test, clusters_test, y_test = prep_data(test)

In [ ]:
# modeling
mrf = MERF(max_iterations=5)
mrf.fit(X_train, Z_train, clusters_train, y_train)

INFO     [merf.py:307] Training GLL is -699.7504668421072 at iteration 1.


In [ ]:
# prediction
y_pred = mrf.predict(X_test, Z_test, clusters_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))
rms

SHAP

In [ ]:
!pip install -q shap

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(mrf.trained_fe_model)

In [ ]:
# reshuffle so not taking just a few participants
X_train = X_train.sample(frac = 1)
X_train = X_train.reset_index(drop = True)

In [ ]:
# shap values
shap_values = explainer.shap_values(X_train[1:100])

In [ ]:
# feature importance
shap.summary_plot(shap_values, X_train[1:100], plot_type="bar")